In [ ]:
import pyodbc as pyo
import time

In [ ]:
import subprocess
import time

import subprocess
import time
import threading

def print_time_elapsed(start_time):
    while True:
        elapsed_time = time.time() - start_time
        print(f"Đã chạy được: {elapsed_time:.2f} giây", end='\r')
        time.sleep(1)

def run_notebook(notebook_path):
    print(f"Bắt đầu chạy kéo dữ liệu nhà thầu {selected_choice}")

    start_time = time.time()  

    time_thread = threading.Thread(target=print_time_elapsed, args=(start_time,), daemon=True)
    time_thread.start()

    try:
        result = subprocess.run(
            ["jupyter", "nbconvert", "--to", "notebook", "--execute", "--inplace", notebook_path],
            check=True,
            capture_output=True, 
            text=True  
        )

        end_time = time.time()
        elapsed_time = end_time - start_time 
        print("\nNotebook đã chạy xong.") 
    except subprocess.CalledProcessError as e:
        print(f"Lỗi khi chạy notebook: {e}")
        print(f"Error output: {e.stderr}")

In [ ]:
import ipynb
print('Please make sure to use LAN connection to do this job')
print('-----------------------------------------------------------------')
print('')
Del_table = input(str('Do you want to delete table TAIMX_SRC_DU_LIEU_GOI_THAU_OVERVIEW before crawling new data(Yes/No): '))
if Del_table.strip().lower() == 'yes':
    connection = pyo.connect(driver='{ODBC Driver 17 for SQL Server}'
                        ,host= 'DETL-DB-VM-P1\POWERPIVOT'
                        ,database='SME'
                        ,trusted_connection='yes')
    cursor = connection.cursor()
    delete_sql = "Delete from SME..TAIMX_SRC_DU_LIEU_GOI_THAU_OVERVIEW"
    cursor.execute(delete_sql)
    connection.commit()
    print("Deleted all the data successfully!!!!")
    print('')   
choice_dict = {
    1: "Đang mở thầu",
    2: "Đang xét thầu",
    3: "Đã trúng thầu",
    4: "All",
    5: "Skip this step"
}

print('What do you want to crawl?? \n 1.--->   Đang mở thầu \n 2.--->   Đang xét thầu \n 3.--->   Đã trúng thầu \n 4.--->   All \n 5.--->   Skip this step')

crawl_type = None
selected_choice = None

while True:
    try:
        crawl_type = int(input('Please Enter Your Choice (1-5): '))
        if crawl_type in choice_dict:
            selected_choice = choice_dict[crawl_type]
            break
        else:
            print('INCORRECT!!!! Your choice is invalid, please make it again!!')
    except ValueError:
        print('INCORRECT!!!! Please enter a valid number between 1 and 5.')

print('   ------------------------------------------------->     CORRECT!!!! YOUR CHOICE IS', selected_choice)
print('   ')

if crawl_type == 3:
    run_notebook('KEO_SRC_GOI_THAU_TRUNG_THAU.ipynb')
if crawl_type == 2:
    run_notebook('KEO_SRC_GOI_THAU_XET_THAU.ipynb')
if crawl_type == 1:
    run_notebook('KEO_SRC_GOI_THAU_MO_THAU.ipynb')
if crawl_type == 4:
    run_notebook('KEO_SRC_GOI_THAU_TRUNG_THAU.ipynb')
    run_notebook('KEO_SRC_GOI_THAU_XET_THAU.ipynb')
    run_notebook('KEO_SRC_GOI_THAU_MO_THAU.ipynb')
if crawl_type == 5:   
    SRC_to_DIM = input(str('Do you want to insert data to table SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW (Yes/No): '))
    if SRC_to_DIM == 'Yes':
        connection = pyo.connect(driver='{ODBC Driver 17 for SQL Server}'
                            ,host= 'DETL-DB-VM-P1\POWERPIVOT'
                            ,database='SME'
                            ,trusted_connection='yes')
        cursor = connection.cursor()
        Drop_query = "Drop table if exists SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW_BACKUP Select * into SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW_BACKUP from SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW  "
        cursor.execute(Drop_query)
        connection.commit()
        insert_query = "Insert into SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW Select Case when Len(Ma_TBMT) <= 15 then Left(Ma_TBMT,12) Else Substring(Ma_TBMT,31,12) End as Ma_TBMT,Ten_goi_thau,Ben_moi_thau,CAST(Ngay_thong_bao as date) as Ngay_TBMT,Linh_vuc,Dia_diem_thau,Ngay_dong_thau,Hinh_thuc_du_thau,DN_trung_thau,Cast(replace(Gia_trung_thau,',','') as float) as Gia_trung_thau,Ngay_phe_duyet as Ngay_duyet_KQDT,Link_chi_tiet,Trang_thai_goi_thau,Ngay_keo_du_lieu,'0' as Remark From (Select *,ROW_NUMBER() Over(Partition by Case when Len(Ma_TBMT) <= 15 then Left(Ma_TBMT,12) Else Substring(Ma_TBMT,31,12) End order by Ngay_keo_du_lieu desc) as RK From SME..TAIMX_SRC_DU_LIEU_GOI_THAU_OVERVIEW) T Where T.RK = 1      "
        cursor.execute(insert_query)
        connection.commit()
        default_value = "Update A Set A.Remark = B.Remark From SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW A,(Select * from SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW where Remark = '1') B Where A.Ma_TBMT = B.Ma_TBMT and A.Trang_thai_goi_thau = B.Trang_thai_goi_thau "
        cursor.execute(default_value)
        connection.commit()
        duplicated_value = "DELETE T FROM (SELECT *,DupRank = ROW_NUMBER() OVER (PARTITION BY Ma_TBMT ORDER BY Ngay_keo_du_lieu desc) FROM SME..TAIMX_DIM_DU_LIEU_GOI_THAU_OVERVIEW ) AS T WHERE DupRank > 1"
        print('DONE!!!')
        cursor.execute(duplicated_value)
        connection.commit()